In [1]:
import os
from bs4 import BeautifulSoup

In [4]:
os.chdir('..')
os.getcwd()

'/workspaces/LW15-Functional-Screening'

In [18]:
# Define the paths to the survival curves directories
significant_curves_path = 'TCGA/survival_curves/significant'
non_significant_curves_path = 'TCGA/survival_curves/non-significant'

# Create a mapping of gene names to their respective survival curve image and significance
all_curves = {}
for folder, significance in [(significant_curves_path, 'Yes'), (non_significant_curves_path, 'No')]:
    for filename in os.listdir(folder):
        gene_name = os.path.splitext(filename)[0].replace('survival_curve_', '').upper()
        all_curves[gene_name] = (os.path.join(folder, filename), significance)

# Load the HTML file
with open('index.html', 'r') as file:
    soup = BeautifulSoup(file, 'html.parser')

# Find the table in the HTML
table = soup.find('table', id='functionalScreening')

# Add new headers for survival curve and significance
headers = table.find('thead').find('tr')
headers.append(soup.new_tag('th', text='Survival Curve'))
headers.append(soup.new_tag('th', text='Survival Curve Significant?'))

In [ ]:
# Add the new columns to each row in the table
for row in table.find('tbody').find_all('tr'):
    gene_name_cell = row.find('td')
    gene_name = gene_name_cell.text.strip().upper()  # Normalize to uppercase for matching

    # Create new cells for the survival curve and significance
    curve_cell = soup.new_tag('td')
    significance_cell = soup.new_tag('td')

    if gene_name in all_curves:
        # Add the survival curve image with class 'modal-trigger' to trigger the modal
        img_tag = soup.new_tag('img', src=all_curves[gene_name][0])
        img_tag['width'] = '220'
        img_tag['height'] = '190'
        img_tag['class'] = 'thumbnail modal-trigger'  # Added 'modal-trigger' class
        img_tag['data-img-src'] = all_curves[gene_name][0]  # Add the data attribute for modal image source
        curve_cell.append(img_tag)
        
        # Set the significance
        significance_cell.string = all_curves[gene_name][1]
    else:
        # No curve for this gene
        curve_cell.string = 'NA'
        significance_cell.string = 'NA'

    # Add the cells to the row
    row.append(curve_cell)
    row.append(significance_cell)

# Write the updated HTML to a new file
with open('updated_index.html', 'w') as file:
    file.write(str(soup.prettify()))